# AADT

## Imports

In [53]:
import urllib.request, json
import pandas as pd
import math
import pickle
import collections
from tqdm import tqdm
import geopandas
import datetime
import os
import numpy as np

## Global Variables

In [54]:
SITE_ID = 2
CLEAN_REPORT_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/clean_report.csv"
TIMESTAMP = 0 # Using random timestamp at the moment

## Generic Functions

## Loading Data

In [55]:
# Get Report Data
df = pd.read_csv('/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/clean_report.csv')
df.head()

,Unnamed: 0,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
0,0,A1M/2259B,2,2019-06-01T00:00:00,00:14:00,0,24,6,6,16,62.0,52,2019-06-01 00:14:00,0.259554,0.234373,0.445363,0.564074,0.325197
1,1,A1M/2259B,2,2019-06-01T00:00:00,00:29:00,1,17,3,7,20,60.0,47,2019-06-01 00:29:00,0.183851,0.117186,0.519590,0.705092,0.293928
2,2,A1M/2259B,2,2019-06-01T00:00:00,00:44:00,2,23,3,3,17,61.0,46,2019-06-01 00:44:00,0.248740,0.117186,0.222681,0.599329,0.287674
3,3,A1M/2259B,2,2019-06-01T00:00:00,00:59:00,3,12,2,0,19,59.0,33,2019-06-01 00:59:00,0.129777,0.078124,0.000000,0.669838,0.206375
4,4,A1M/2259B,2,2019-06-01T00:00:00,01:14:00,4,14,5,3,16,61.0,38,2019-06-01 01:14:00,0.151407,0.195311,0.222681,0.564074,0.237644


## Median Model
A median model, similar to the UK Department of Transport’s method [40], is developed.
From January 2019 to April 2022 Traffic Count data is obtained. Normalised Traffic
Counts are taken by dividing each Traffic Count by the mean Traffic Count over the
entire date range for that site. At time, $t^∗$, the median of Normalised Traffic Counts
is taken over all sites.

More formally, let $C_{i,t}$ be the observed Traffic Count at site i at time t. Let D denote
the date range mentioned. Let S refer to the set of count sites. This gives:
$$N(t^∗) = median_{i∈S}( \frac{C_{i,t^*}}{mean_{t \in D}(C_{i,t})})$$

In [56]:
# Groupby sites
dfs = [site for _, site in df.groupby('site_id')]

In [57]:
df_site = df.loc[df['site_id'] == SITE_ID]
mean_count = df_site['total_volume'].mean()
print("site_id: {}, \nmean_count: {}".format(SITE_ID, mean_count))

site_id: 2, 
mean_count: 159.90308320215664


In [58]:
# Random index as an example
random_t = df.iloc[np.random.randint(0, len(df))]['timestamp']
random_t

df_t = df.loc[df['timestamp'] == random_t]
median_count = df_t['total_volume'].median()
print("len(df_t): {}, \nrandom_t: {}, \nmedian_count: {}".format(len(df_t), random_t, median_count))

len(df_t): 3, 
random_t: 2020-01-16 02:29:00, 
median_count: 27.0


In [ ]:
# Random traffic count as an example
random_traffic_count = df.iloc[np.random.randint(0, len(df))]['total_volume']

# Normalised Traffic Count
